In [1]:
import pandas as pd
import os
import glob
import numpy as np
import underthesea as uts
import transformers as tfm
import numpy as np
import matplotlib.pyplot as plt
import re
import string

In [2]:
path = os.getcwd()
datapath = glob.glob(os.path.join(path,"data/*.txt"))
fulldata = []
categorydata = []
for i in datapath:
    filename = os.path.basename(i).replace(".txt","")
    with open(i,'r',encoding='utf-8') as f:
#     lines = [i.strip() for i in f.readlines()]
        lines = f.readlines()
    for j in lines:
        fulldata.append(j)
        categorydata.append(filename)

print(fulldata[:5])
print(len(fulldata))

['Thích đánh lộn không? __eou__ Ngon nhà vô __eou__\n', 'Solo yasua không __eou__ Chấp lun 2 mạng đầu __eou__\n', 'Mai đi picnic không? __eou__ Mai bận học rồi __eou__\n', 'Mai học ca mấy vậy? __eou__ Mai học ca 3 __eou__\n', 'Còn tiền không? __eou__ Còn chết liền __eou__\n']
5855


In [3]:
data = [i.split('__eou__') for i in fulldata]
print(data[:5])
data_question = [i[0] for i in data]
data_answer = [i[1] for i in data]
print(data_question[:5])

[['Thích đánh lộn không? ', ' Ngon nhà vô ', '\n'], ['Solo yasua không ', ' Chấp lun 2 mạng đầu ', '\n'], ['Mai đi picnic không? ', ' Mai bận học rồi ', '\n'], ['Mai học ca mấy vậy? ', ' Mai học ca 3 ', '\n'], ['Còn tiền không? ', ' Còn chết liền ', '\n']]
['Thích đánh lộn không? ', 'Solo yasua không ', 'Mai đi picnic không? ', 'Mai học ca mấy vậy? ', 'Còn tiền không? ']


In [4]:
data_table = pd.DataFrame({'question':data_question,'answer':data_answer,'category':categorydata},)
data_table = data_table.dropna()
data_table.head()

,question,answer,category
0,Thích đánh lộn không?,Ngon nhà vô,bạn bè
1,Solo yasua không,Chấp lun 2 mạng đầu,bạn bè
2,Mai đi picnic không?,Mai bận học rồi,bạn bè
3,Mai học ca mấy vậy?,Mai học ca 3,bạn bè
4,Còn tiền không?,Còn chết liền,bạn bè


In [5]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    return text

In [6]:
data_table['question'] = data_table['question'].apply(clean_text)
data_table['answer'] = data_table['answer'].apply(clean_text)
data_table.head()

,question,answer,category
0,thích đánh lộn không,ngon nhà vô,bạn bè
1,solo yasua không,chấp lun 2 mạng đầu,bạn bè
2,mai đi picnic không,mai bận học rồi,bạn bè
3,mai học ca mấy vậy,mai học ca 3,bạn bè
4,còn tiền không,còn chết liền,bạn bè


In [7]:
stop_words = ['bị','bởi','cả','các','cái','cần','càng','chỉ','chiếc','cho','chứ','chưa','có','có_thể','cứ','cùng','cũng','đã','đang','để','do','đó','được','gì','khi','không','là','lại','lên','lúc','mà','mỗi','này','nên','nếu','ngay','nhiều','như','nhưng','những','nơi','nữa','phải','qua','ra','rằng','rất','rồi','sau','sẽ','theo','thì','từ','từng','và','vẫn','vào','vậy','vì','việc','với']

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text
    
data_table['message_clean'] = data_table['question'].apply(remove_stopwords)
data_table.head()

,question,answer,category,message_clean
0,thích đánh lộn không,ngon nhà vô,bạn bè,thích đánh lộn
1,solo yasua không,chấp lun 2 mạng đầu,bạn bè,solo yasua
2,mai đi picnic không,mai bận học rồi,bạn bè,mai đi picnic
3,mai học ca mấy vậy,mai học ca 3,bạn bè,mai học ca mấy
4,còn tiền không,còn chết liền,bạn bè,còn tiền


In [9]:
data_table['merge'] = data_table['question'] + data_table['answer']
data_table.head()

,question,answer,category,message_clean,merge
0,thích đánh lộn không,ngon nhà vô,bạn bè,thích đánh lộn,thích đánh lộn không ngon nhà vô
1,solo yasua không,chấp lun 2 mạng đầu,bạn bè,solo yasua,solo yasua không chấp lun 2 mạng đầu
2,mai đi picnic không,mai bận học rồi,bạn bè,mai đi picnic,mai đi picnic không mai bận học rồi
3,mai học ca mấy vậy,mai học ca 3,bạn bè,mai học ca mấy,mai học ca mấy vậy mai học ca 3
4,còn tiền không,còn chết liền,bạn bè,còn tiền,còn tiền không còn chết liền


In [10]:
data_table['question_split'] = data_table['question'].apply(uts.pos_tag)
data_table['merge_split'] = data_table['merge'].apply(uts.pos_tag)
data_table['answer_split'] = data_table['answer'].apply(uts.pos_tag)
data_table = data_table.dropna()
data_table.head(10)

,question,answer,category,message_clean,merge,question_split,merge_split,answer_split
0,thích đánh lộn không,ngon nhà vô,bạn bè,thích đánh lộn,thích đánh lộn không ngon nhà vô,"[(thích, V), (đánh lộn, V), (không, R)]","[(thích, V), (đánh lộn, V), (không, R), (ngon,...","[(ngon, A), (nhà, N), (vô, V)]"
1,solo yasua không,chấp lun 2 mạng đầu,bạn bè,solo yasua,solo yasua không chấp lun 2 mạng đầu,"[(solo, N), (yasua, V), (không, R)]","[(solo, N), (yasua, V), (không, R), (chấp, V),...","[(chấp, V), (lun, N), (2, M), (mạng, N), (đầu,..."
2,mai đi picnic không,mai bận học rồi,bạn bè,mai đi picnic,mai đi picnic không mai bận học rồi,"[(mai, N), (đi, V), (picnic, V), (không, R)]","[(mai, N), (đi, V), (picnic, V), (không, R), (...","[(mai, N), (bận, V), (học, V), (rồi, T)]"
3,mai học ca mấy vậy,mai học ca 3,bạn bè,mai học ca mấy,mai học ca mấy vậy mai học ca 3,"[(mai, N), (học, V), (ca, N), (mấy, L), (vậy, P)]","[(mai, N), (học, V), (ca, N), (mấy, L), (vậy, ...","[(mai, N), (học, V), (ca, N), (3, M)]"
4,còn tiền không,còn chết liền,bạn bè,còn tiền,còn tiền không còn chết liền,"[(còn, R), (tiền, N), (không, R)]","[(còn, R), (tiền, N), (không, R), (còn, R), (c...","[(còn, R), (chết, V), (liền, R)]"
5,mấy giờ rồi nhỉ,1100 rồi,bạn bè,mấy giờ nhỉ,mấy giờ rồi nhỉ 1100 rồi,"[(mấy, L), (giờ, N), (rồi, T), (nhỉ, T)]","[(mấy, L), (giờ, N), (rồi, C), (nhỉ, T), (1100...","[(1100, N), (rồi, T)]"
6,bao lâu rồi chưa về quê,1 tháng rồi,bạn bè,bao lâu về quê,bao lâu rồi chưa về quê 1 tháng rồi,"[(bao lâu, P), (rồi, C), (chưa, R), (về, V), (...","[(bao lâu, P), (rồi, C), (chưa, R), (về, V), (...","[(1, M), (tháng, N), (rồi, M)]"
7,bạn có crush chưa,mình có rồi,bạn bè,bạn crush,bạn có crush chưa mình có rồi,"[(bạn, N), (có, V), (crush, N), (chưa, R)]","[(bạn, N), (có, V), (crush, N), (chưa, R), (mì...","[(mình, P), (có, V), (rồi, T)]"
8,crush bao lâu rồi,khoảng 10 năm,bạn bè,crush bao lâu,crush bao lâu rồi khoảng 10 năm,"[(crush, N), (bao lâu, P), (rồi, T)]","[(crush, N), (bao lâu, P), (rồi, C), (khoảng, ...","[(khoảng, N), (10, M), (năm, M)]"
9,crush có bồ chưa,có rồi huhu,bạn bè,crush bồ,crush có bồ chưa có rồi huhu,"[(crush, N), (có, V), (bồ, N), (chưa, R)]","[(crush, N), (có, V), (bồ, N), (chưa, R), (có,...","[(có, V), (rồi, T), (huhu, M)]"


In [11]:
def pos_merge(sentence):
    sentence = ["/".join(i) for i in sentence]
    return sentence

In [12]:
data_table['question_split'] = [[ "/".join(i) for i in sentence] for sentence in data_table['question_split']]
data_table['merge_split'] = [[ "/".join(i) for i in sentence] for sentence in data_table['merge_split']]
data_table['answer_split'] = [[ "/".join(i) for i in sentence] for sentence in data_table['answer_split']]
data_table.head(10)

,question,answer,category,message_clean,merge,question_split,merge_split,answer_split
0,thích đánh lộn không,ngon nhà vô,bạn bè,thích đánh lộn,thích đánh lộn không ngon nhà vô,"[thích/V, đánh lộn/V, không/R]","[thích/V, đánh lộn/V, không/R, ngon/A, nhà/N, ...","[ngon/A, nhà/N, vô/V]"
1,solo yasua không,chấp lun 2 mạng đầu,bạn bè,solo yasua,solo yasua không chấp lun 2 mạng đầu,"[solo/N, yasua/V, không/R]","[solo/N, yasua/V, không/R, chấp/V, lun/N, 2/M,...","[chấp/V, lun/N, 2/M, mạng/N, đầu/N]"
2,mai đi picnic không,mai bận học rồi,bạn bè,mai đi picnic,mai đi picnic không mai bận học rồi,"[mai/N, đi/V, picnic/V, không/R]","[mai/N, đi/V, picnic/V, không/R, mai/N, bận/V,...","[mai/N, bận/V, học/V, rồi/T]"
3,mai học ca mấy vậy,mai học ca 3,bạn bè,mai học ca mấy,mai học ca mấy vậy mai học ca 3,"[mai/N, học/V, ca/N, mấy/L, vậy/P]","[mai/N, học/V, ca/N, mấy/L, vậy/P, mai/N, học/...","[mai/N, học/V, ca/N, 3/M]"
4,còn tiền không,còn chết liền,bạn bè,còn tiền,còn tiền không còn chết liền,"[còn/R, tiền/N, không/R]","[còn/R, tiền/N, không/R, còn/R, chết/V, liền/R]","[còn/R, chết/V, liền/R]"
5,mấy giờ rồi nhỉ,1100 rồi,bạn bè,mấy giờ nhỉ,mấy giờ rồi nhỉ 1100 rồi,"[mấy/L, giờ/N, rồi/T, nhỉ/T]","[mấy/L, giờ/N, rồi/C, nhỉ/T, 1100/A, rồi/T]","[1100/N, rồi/T]"
6,bao lâu rồi chưa về quê,1 tháng rồi,bạn bè,bao lâu về quê,bao lâu rồi chưa về quê 1 tháng rồi,"[bao lâu/P, rồi/C, chưa/R, về/V, quê/N]","[bao lâu/P, rồi/C, chưa/R, về/V, quê/N, 1/M, t...","[1/M, tháng/N, rồi/M]"
7,bạn có crush chưa,mình có rồi,bạn bè,bạn crush,bạn có crush chưa mình có rồi,"[bạn/N, có/V, crush/N, chưa/R]","[bạn/N, có/V, crush/N, chưa/R, mình/P, có/V, r...","[mình/P, có/V, rồi/T]"
8,crush bao lâu rồi,khoảng 10 năm,bạn bè,crush bao lâu,crush bao lâu rồi khoảng 10 năm,"[crush/N, bao lâu/P, rồi/T]","[crush/N, bao lâu/P, rồi/C, khoảng/A, 10/M, nă...","[khoảng/N, 10/M, năm/M]"
9,crush có bồ chưa,có rồi huhu,bạn bè,crush bồ,crush có bồ chưa có rồi huhu,"[crush/N, có/V, bồ/N, chưa/R]","[crush/N, có/V, bồ/N, chưa/R, có/V, rồi/T, huh...","[có/V, rồi/T, huhu/M]"


In [13]:
from gensim.models import Word2Vec

model_q2q = Word2Vec(vector_size=100,min_count=1)
model_q2q.build_vocab(data_table['question_split'])
model_q2q.train(data_table['question_split'],total_examples=model_q2q.corpus_count, epochs=model_q2q.epochs)

model_q2a = Word2Vec(vector_size=100,min_count=1)
model_q2a.build_vocab(data_table['merge_split'])
model_q2a.train(data_table['merge_split'],total_examples=model_q2a.corpus_count, epochs=model_q2a.epochs)

(308596, 425640)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

In [15]:
def get_embedding(sentence,model,pos=False):
    sentence = sentence.lower()
    sentence = clean_text(sentence)
    P = np.zeros(100)
    num = 0
    if pos==True:
        word = uts.pos_tag(sentence)
        word = pos_merge(word)
    else:
        word = uts.word_tokenize(sentence)
    for i in word:
        try:
            embed = model.wv[i]
        except:
            continue
        else:
            P += embed
            num +=1
    if(num == 0):
        return P
    else:
        P /= num
        return P

def get_question(question):
    question_embedding = get_embedding(question,model_q2q,pos=True)
    question_list = []
    for i in data_table['question_embedding']:
        question_list.append(cosine_similarity(np.expand_dims(question_embedding,0), np.expand_dims(i,0)))
    max_similarity = np.argmax(question_list, axis=None)
    return data_table['question'][max_similarity]

def get_answer_raw(question):
    question_embedding = get_embedding(get_question(question),model_q2q,pos=True)
    answer_list = []
    for i in data_table['question_embedding']:
        answer_list.append(cosine_similarity(np.expand_dims(question_embedding,0), np.expand_dims(i,0)))
    max_similarity = np.argmax(answer_list, axis=None)
    
    return data_table['answer'][max_similarity]
    
def get_answer_q2a(question):
    question_embedding = get_embedding(get_question(question),model_q2a,pos=True)
    answer_list = []
    for i in data_table['answer_embedding']:
        answer_list.append(cosine_similarity(np.expand_dims(question_embedding,0), np.expand_dims(i,0)))
    max_similarity = np.argmax(answer_list, axis=None)
    
    return data_table['answer'][max_similarity]

In [16]:
data_table['question_embedding'] = [get_embedding(i,model_q2q,pos=True) for i in data_table['question']]
data_table['answer_embedding'] = [get_embedding(i,model_q2a,pos=True) for i in data_table['answer']]
data_table.head()

,question,answer,category,message_clean,merge,question_split,merge_split,answer_split,question_embedding,answer_embedding
0,thích đánh lộn không,ngon nhà vô,bạn bè,thích đánh lộn,thích đánh lộn không ngon nhà vô,"[thích/V, đánh lộn/V, không/R]","[thích/V, đánh lộn/V, không/R, ngon/A, nhà/N, ...","[ngon/A, nhà/N, vô/V]","[-0.05416531728891035, 0.13219763152301311, -0...","[-0.14206348111232123, 0.2142828404903412, 0.0..."
1,solo yasua không,chấp lun 2 mạng đầu,bạn bè,solo yasua,solo yasua không chấp lun 2 mạng đầu,"[solo/N, yasua/V, không/R]","[solo/N, yasua/V, không/R, chấp/V, lun/N, 2/M,...","[chấp/V, lun/N, 2/M, mạng/N, đầu/N]","[-0.035589702834840864, 0.0853033997118473, -0...","[-0.09917506016790867, 0.12875984949059785, 0...."
2,mai đi picnic không,mai bận học rồi,bạn bè,mai đi picnic,mai đi picnic không mai bận học rồi,"[mai/N, đi/V, picnic/V, không/R]","[mai/N, đi/V, picnic/V, không/R, mai/N, bận/V,...","[mai/N, bận/V, học/V, rồi/T]","[-0.0556608943734318, 0.12820010192808695, -0....","[-0.15255676209926605, 0.24424213729798794, 0...."
3,mai học ca mấy vậy,mai học ca 3,bạn bè,mai học ca mấy,mai học ca mấy vậy mai học ca 3,"[mai/N, học/V, ca/N, mấy/L, vậy/P]","[mai/N, học/V, ca/N, mấy/L, vậy/P, mai/N, học/...","[mai/N, học/V, ca/N, 3/M]","[-0.05406220573931932, 0.14169600568711757, -0...","[-0.15507538057863712, 0.27723681554198265, 0...."
4,còn tiền không,còn chết liền,bạn bè,còn tiền,còn tiền không còn chết liền,"[còn/R, tiền/N, không/R]","[còn/R, tiền/N, không/R, còn/R, chết/V, liền/R]","[còn/R, chết/V, liền/R]","[-0.08682342246174812, 0.1936942587296168, -0....","[-0.09963977802544832, 0.14636657324930033, 0...."


In [17]:
questionpack = ["cậu là sinh viên năm mấy","cậu đang ở đâu","cậu có ổn không","đang làm gì vậy","có đang crush ai không"]
for i in questionpack:
    print("Câu hỏi: ", i)
    print("Câu hỏi trong data: ",get_question(i))
    print("Trả lời nguyên bản: ",get_answer_raw(i))
    print("Trả lời dùng merge: ",get_answer_q2a(i))
    print("----------------")
    
# answer_list = [get_answer_raw(i) for i in data_table['question']]
# print(accuracy_score(answer_list,data_table['answer']))

Câu hỏi:  cậu là sinh viên năm mấy
Câu hỏi trong data:  bạn là sinh viên năm thứ mấy 
Trả lời nguyên bản:   sinh viên năm thứ 3 
Trả lời dùng merge:   mình là sinh viên năm 3 
----------------
Câu hỏi:  cậu đang ở đâu
Câu hỏi trong data:  hiện giờ cậu đang ở đâu 
Trả lời nguyên bản:   tớ đang ở nhật 
Trả lời dùng merge:   mình đang ở cùng gia đình ở quận 6 
----------------
Câu hỏi:  cậu có ổn không
Câu hỏi trong data:  cậu có khỏe không 
Trả lời nguyên bản:   tớ rất khỏe 
Trả lời dùng merge:   không có nha 
----------------
Câu hỏi:  đang làm gì vậy
Câu hỏi trong data:  bạn đang làm gì vậy  
Trả lời nguyên bản:   có 
Trả lời dùng merge:   mình đang làm nhân viên văn phòng 
----------------
Câu hỏi:  có đang crush ai không
Câu hỏi trong data:  hiện giờ có đang thích ai không 
Trả lời nguyên bản:   hiện tại có crush một bạn trong lớp anh văn 
Trả lời dùng merge:   mình có thích chơi đàn và hiện tại vẫn đang luyện tập 
----------------


In [18]:
questiongenerator = data_table.sample(n=5)
for i in questiongenerator['question']:
    print("Câu hỏi: ", i)
    print("Trả lời nguyên bản: ",get_answer_raw(i))
    print("Trả lời dùng merge: ",get_answer_q2a(i))
    print("----------------")

Câu hỏi:  thế giữa phim hoạt hình và hành động bắn súng tâm lý tình cảm cậu thích xem gì 
Trả lời nguyên bản:   gì cũng được miễn là xem cùng cậu hihi 
Trả lời dùng merge:   sở thích của mình là chơi game và xem phim 
----------------
Câu hỏi:  chỗ bạn trọ họ có bao ăn không 
Trả lời nguyên bản:   chỗ mình trọ không có bao ăn nah 
Trả lời dùng merge:   chỗ mình trọ không có bao ăn nah 
----------------
Câu hỏi:  lâu rồi không về thăm nhà bạn gia đình ở quê có khoẻ không  
Trả lời nguyên bản:   cảm ơn bạn ba mẹ mình vẫn khỏe tết này nếu có dịp bạn hãy qua nhà mình chơi nhé 
Trả lời dùng merge:   mình sẽ về quê á 
----------------
Câu hỏi:  tâm trạng cảu bạn như thế nào 
Trả lời nguyên bản:   rất là tốt 
Trả lời dùng merge:   bạn mún mua laptop như thế nào 
----------------
Câu hỏi:  bạn đang học ngành gì 
Trả lời nguyên bản:   mình học công nghệ thông tin 
Trả lời dùng merge:   mình đang học làm con gáiahihi 
----------------


In [33]:
model_q2a.wv.vectors.shape

(6140, 100)